In [6]:
# import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn

import pandas as pd
import numpy as np
from ta.volatility import BollingerBands
from ta.trend import MACD
from ta.momentum import RSIIndicator

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.metrics import mean_squared_error
from torch.utils.data import TensorDataset, DataLoader

In [14]:
data = pd.read_csv('BTCUSD.csv')
data = data.drop(['MA_daily','RSI','BB_High','BB_Low','BBW','MACD','Breakout','MA_weekly'], axis =1).dropna()
data

,time,open,high,low,close,tick_volume,RSI_signal,BB_signal,MACD_signal,MA_signal,Engulfing,Consolidation,ATR
13,2017-03-29 14:30:00,1041.010,1042.000,1041.010,1042.000,12,0,0,0,0,0,0,0.602429
14,2017-03-29 20:15:00,1042.000,1042.000,1041.502,1041.502,15,0,0,0,0,0,0,0.630857
15,2017-03-29 20:30:00,1041.502,1041.502,1041.502,1041.502,24,0,0,0,0,0,0,0.610857
16,2017-03-29 20:45:00,1044.731,1044.731,1041.500,1041.509,46,0,0,0,0,0,0,0.731286
17,2017-03-29 21:00:00,1041.506,1042.000,1041.506,1042.000,68,0,0,0,0,0,0,0.766429
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213661,2023-07-03 14:00:00,30642.738,30698.738,30637.681,30678.538,555,0,1,-1,1,0,0,45.078500
213662,2023-07-03 14:15:00,30678.538,30696.738,30655.838,30656.538,404,0,0,-1,-1,0,0,46.121357
213663,2023-07-03 14:30:00,30656.538,30753.438,30641.038,30746.038,606,0,1,1,1,0,0,52.021357
213664,2023-07-03 14:45:00,30746.038,30748.738,30695.022,30728.638,555,0,1,1,1,0,0,52.999429


In [13]:
# device agnostic code.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [15]:
# Define the sequence length
sequence_length = 100

# Normalization
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data.values[:, 1:]) 

# Convert pandas DataFrame to Tensor
data_normalized = torch.FloatTensor(data_normalized).to(device)

# Creating sequences
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1, data.columns.get_loc('close') - 1]  # Adjusted for 0-indexing
        inout_seq.append((train_seq ,train_label))
    return inout_seq

train_size = int(len(data_normalized) * 0.8)
train_data = data_normalized[:train_size]
test_data = data_normalized[train_size:]

train_sequence = create_inout_sequences(train_data, sequence_length)
test_sequence = create_inout_sequences(test_data, sequence_length)



In [16]:
# Creating dataloaders
batch_size = 128  # Feel free to adjust this

train_dataloader = DataLoader(train_sequence, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_sequence, batch_size=batch_size, shuffle=False)


In [18]:
# Create LSTM model
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_layer_size=256, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, batch_first=True)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size).to(device),
                            torch.zeros(1,1,self.hidden_layer_size).to(device))
    def init_hidden(self, batch_size):
        
        return (torch.zeros(1, batch_size, self.hidden_layer_size).to(device),
            torch.zeros(1, batch_size, self.hidden_layer_size).to(device))

    def forward(self, input_seq):

        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions.view(input_seq.size(0), -1)



# Instantiate the model, define the loss and the optimizer
model = LSTM(input_size = 11).to(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [14]:
# device agnostic code.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [19]:
# Training loop
epochs = 50

for i in range(epochs):
    for seq, labels in train_dataloader:
        optimizer.zero_grad()
        model.hidden_cell = model.init_hidden(seq.size(0))  # initialize with batch size

        y_pred = model(seq.to(device))

        single_loss = loss_function(y_pred, labels.to(device))
        single_loss.backward()
        optimizer.step()

    if i%25 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

RuntimeError: input.size(-1) must be equal to input_size. Expected 11, got 1200